---
title: "A walkthrough: the American Declaration of Independence"
---

```{julia}
#| output: false
#| echo: false
fullinfo = true


# Set up Julia environment:
using Pkg
pkglist = [
    "CitableBase", 
    "CitableText", "CitableObject", "CitableImage",
    "CitableCorpus",

    "Orthography", "CitableTeiReaders", "EditionBuilders",
    "CitablePhysicalText",

    "VectorAlignments", "StatsBase", "OrderedCollections"
]
for p in pkglist
    Pkg.add(p)    
end

using Markdown

if fullinfo
    allpkgs = map(collect(keys(Pkg.dependencies()))) do uukey
        Pkg.dependencies()[uukey]
    end
    pkglistinfo = filter(pi -> pi.name in pkglist, allpkgs) 
    pkglist_md = map(p -> string("- ", p.name, " ", p.version), pkglistinfo) |> sort
else
    pkglist_md = "TBA"
end
```





::: {.callout-tip collapse="true"}
`{julia} string("## See versions of Julia packages used on this page","\n\n", "This page was compiled with the following  versions of the packages illustrated here:", "\n\n", join(pkglist_md,"\n")) |> Markdown.parse`
:::

## Background and sources

The historian and classicist Danielle Allen offers a new interpretation of the American Declaration of Independence that is literally based on a new reading: she punctuates the text differently from most printed editions. (See her book [@allen2014] and the subsequent manuscript available from the Institute for Advanced Study [@allen2015].)

This page briefly illustrates how we could apply the Julia packages for citation and for working with digital texts to the problem of reading the text of the Declaration.

### Sources

1.  The U.S. National Archives has published freely available, high-resolution images of both the original signed parchment, and a widely used engraved reproduction. (See the presentation of the Declaration of Independence from the [National Archives](https://www.archives.gov/founding-docs/declaration), with links to [downloadable images](https://www.archives.gov/founding-docs/downloads)) These images have been added to the Homer Multitext project's citable image service.
2.  Two transcriptions of the text in TEI-compliant XML are included in this site's github repository:
    -   [transcription 1](https://raw.githubusercontent.com/neelsmith/quart-home/main/walkthrough/adoi/declaration_engraving.xml): a transcription of the text of the engraving, taken from [this page](https://www.archives.gov/founding-docs/declaration-transcript) of the National Archives web site
    -   [transcription 2](https://raw.githubusercontent.com/neelsmith/quart-home/main/walkthrough/adoi/declaration_original.xml): a transcription of the text from the image of the original parchment
3.  The relations of text, pages and documentary image are documented in a delimited-text file in this site's github repository [here](https://raw.githubusercontent.com/neelsmith/quart-home/main/walkthrough/adoi/declaration.cex).

## Citation with URNs

::: callout-tip
## Learn more about scholarly [citation with URNs](../cite/)
:::



### Citing texts

Cite the text of the original and the engraving using `CtsUrn`s with different version identifiers.





```{julia}
#| output: false
using CitableBase, CitableText
engraving_text = CtsUrn("urn:cts:citeusnatarch:foundingdocuments.declindep.engraving:")
original_text = CtsUrn("urn:cts:citeusnatarch:foundingdocuments.declindep.original:")
```





::: callout-tip
## Learn more about [citation with `CtsUrn`s](../cite/urns/cts/)
:::

### Citing objects

Identify a collection of pages (front and back) for the original and for the engraving using `Cite2Urn`s.





```{julia}
#| output: false
using CitableObject
engraving_pages = Cite2Urn("urn:cite2:citeusnatarch:declindep.engraving:")
original_pages = Cite2Urn("urn:cite2:citeusnatarch:declindep.original:")
```





::: callout-tip
## Learn more about [citation with `Cite2Urn`s](../cite/urns/cite2/)
:::

### Citing images

Identify documentary images for the front of the parchment and the first page of the engraving using `Cite2Urn`s.





```{julia}
#| output: false
original_img1 = Cite2Urn("urn:cite2:citeusnatarch:declindep.v1:Declaration_Pg1of1_AC")
engraving_img1 = Cite2Urn("urn:cite2:citeusnatarch:declindep.v1:Declaration_Engrav_Pg1of1_AC")
```





## Digital texts

### Orthography

Define an orthographic system for the texts we will read.





```{julia}
#| output: false
using Orthography
ortho = simpleAscii()
```





::: callout-tip
## Learn more about [orthography](../digitexts/orthography/)
:::

### Create citable texts

Read the XML source from a URL, and make the marked-up content accessible through canonical citation with the `CitableTeiReaders` package.





```{julia}
#| output: false
using CitableTeiReaders
engraving_url = "https://raw.githubusercontent.com/neelsmith/quart-home/main/walkthrough/adoi/declaration_engraving.xml"
engraving_corpus = readcitable(engraving_url, engraving_text, TEIAnonblock, UrlReader)

original_url = "https://raw.githubusercontent.com/neelsmith/quart-home/main/walkthrough/adoi/declaration_original.xml"
original_corpus = readcitable(original_url, engraving_text, TEIAnonblock, UrlReader)
```





Compose a univocal diplomatic edition from the multivalent XML document with the `EditionBuilders` package.





```{julia}
#| output: false
using EditionBuilders
builder = MidDiplomaticBuilder("Diplomatic edition buidler", "dipl")
engraving_dipl = edited(builder, engraving_corpus; edition = "engraving_dip")
original_dipl = edited(builder, original_corpus; edition = "original_dipl")
```





::: callout-tip
## Learn more about creating citable texts

-   [making markup citable](../digitexts/editions/markupreaders.qmd)
-   [building univocal editions](../digitexts/editions/editionbuilders.qmd)
:::

### A full digital scholarly edition

#### Collect relations of text, image and artifact

Read records relating text, image and artifact using the `CitablePhysicalText` package.





```{julia}
#| output: false
using CitablePhysicalText
delimited_url = "https://raw.githubusercontent.com/neelsmith/quart-home/main/walkthrough/adoi/declaration.cex"
triplesets = fromcex(delimited_url, DSECollection, UrlReader)
```





::: callout-tip
## Learn more about [modelling physical texts](../digitexts/physical/). 
:::

#### Services for images

Configure the Homer Mulitext project's image service using the `CitableImage` package.





```{julia}
#| output: false
using CitableImage
imgbaseurl = "http://www.homermultitext.org/iipsrv"
imgroot = "/project/homer/pyramidal/deepzoom"
imgservice = IIIFservice(imgbaseurl, imgroot)
```





::: callout-tip
## Learn more about [citable image services](../cite/urns/imgs/)
:::



## Application: explore differences in the texts visually

:::{.callout-warning}
## TBA
:::

First, we'll find passages where the two editions differ. One way to focus our attention on differences in content rather than formatting is to compare tokenizations of each text.  The following function compares two citable text passages by tokenizing each and comparing the result.





```{julia}
#| output: false
using CitableCorpus

"""True if text content of tokens in two citable passages matches."""
function tokensmatch(psg1::CitableCorpus.CitablePassage, psg2::CitableCorpus.CitablePassage, orthography::OrthographicSystem)
    tkns1 = tokenize(psg1,ortho)
    tkns2 = tokenize(psg2,ortho)
    text1 = map(t -> t.passage.text, tkns1)
    text2 = map(t -> t.passage.text, tkns2)
    text1 == text2
end
```






Next we use the function to find parallel passages where the texts differ.





```{julia}
#| output: false
@assert length(original_dipl) == length(engraving_dipl.passages)

difflist = []   
for (orig,dipl) in zip(original_dipl.passages, engraving_dipl.passages)
    if ! tokensmatch(orig,dipl,ortho)
        push!(difflist,(orig,dipl))
    end
end
```






Look in appropriate set of triples:





```{julia}
triplesets[1] |> label
```

```{julia}
#| output: false
triples_engraving = triplesets[1]
triples_original = triplesets[2]
```




Now find images





```{julia}
(psg1_orig, psg1_engraving) = difflist[1]
imgs_orig = imagesfortext(psg1_orig.urn, triples_original; keepversion = false)
imgs_engraving = imagesfortext(psg1_engraving.urn, triples_engraving; keepversion = false)
```





Display the result on this web page!





```{julia}
#| as-is: true
ict = "http://www.homermultitext.org/ict2/?"
img_md_original = linkedMarkdownImage(ict, imgs_orig[1], imgservice; w=600, caption="Passage in original")

img_md_engraving = linkedMarkdownImage(ict, imgs_orig[1], imgservice; w=600, caption="Passage in engraving")


"""
 $(img_md_original)

 $(img_md_engraving) 
""" |> Markdown.parse
```





-   manually use the Homer Multitext project's Image Citation Tool to zoom in to more specific region of interest and cite it with a URN. Here we save references to the region of two images illustrating the word "Happiness."


Manually explore, create two more detailed references.





```{julia}
happiness_engraving = Cite2Urn("urn:cite2:citeusnatarch:declindep.v1:Declaration_Engrav_Pg1of1_AC@0.4672,0.1935,0.07188,0.01804")
happiness_original = Cite2Urn("urn:cite2:citeusnatarch:declindep.v1:Declaration_Pg1of1_AC@0.4765,0.1892,0.07596,0.01857")


detail_md_original = linkedMarkdownImage(ict, happiness_original, imgservice; w=600, caption="Passage in original")

detail_md_engraving = linkedMarkdownImage(ict,happiness_engraving, imgservice; w=600, caption="Passage in engraving")


"""
 $(detail_md_original)

 $(detail_md_engraving) 
""" |> Markdown.parse

```





## Application: organize differences in the texts as a sequence alignment


:::{.callout-warning}
## TBA
:::




::: callout-tip
## Learn more about [citable text corpora](../digitexts/editions/citablecorpora.qmd)
:::






```{julia}
using VectorAlignments
tkns1 = split("Now is the time")
tkns2 = split("Now is a time")
featurematrix(tkns1, tkns2)
```







::: callout-tip
## Learn more about [sequence alignments](../utils/)
:::



## Application: basic vocabulary frequency

It is straightforward to find the vocabulary of a citable corpus with a known orthography.





```{julia}
tkns = tokenize(original_dipl, ortho)
lex = filter(tkn -> tokencategory(tkn) isa LexicalToken,tkns)  
vocab = map(tkn -> tokentext(tkn), lex)
```






We can use the `StatsBase` package to count frequencies of items in a list, and can convert that dictionary to a sortable ordered dictionary with the `OrderedCollections`  package.





```{julia}
using StatsBase, OrderedCollections
counts = countmap(vocab) |> OrderedDict
sort!(counts; byvalue = true, rev = true)
```





## Works cited 